<br>

# Introdução

In [3]:
#!pip3 install dash==2.0.0
!pip3 install traquitanas --upgrade
#!pip3 install basedosdados --upgrade
#!pip3 install gerador-endereco --upgrade

  Attempting uninstall: traquitanas
    Found existing installation: traquitanas 1.0.29
    Uninstalling traquitanas-1.0.29:
      Successfully uninstalled traquitanas-1.0.29


In [4]:
import os
import time
import json
import random
import pandas as pd
from traquitanas import personal

In [5]:
personal.classify_name('Ana')

"Ana" é um nome "Feminino" com 99.57000000000001% de certeza


'Feminino'

In [6]:
import basedosdados as bd
#from classify_name import

In [7]:
data_path = os.path.join('..', 'data')

<br>

## Lista de Nomes

In [8]:
# Nomes
df = bd.read_sql(
    """
    SELECT nome FROM `basedosdados.br_cgu_servidores_executivo_federal.servidores_civis_cadastro` LIMIT 100000
    """,
    billing_project_id='my-vps-291912',
)
display(df.head(5))

# Registros Únicos
list_nomes = list(set(list(df['nome'])))
print(list_nomes[0:5])

Downloading: 100%|█████████████████| 100000/100000 [00:02<00:00, 38711.88rows/s]


,nome
0,JOSE WILAMES FREITAS
1,JOSE WILAMES FREITAS
2,JOSE WILAMOR DE SOUZA MOTA
3,JOSE WILBERT DE LIMA
4,JOSE WILDE DE OLIVEIRA CABRAL


['JOYCE CRAVEIRO MELO DE CARVALHO', 'LUCINEIA FATIMA DA SILVA', 'LUIZ CARLOS DOS SANTOS ROCHA', 'MARCELO HITOSHI MIYANO', 'JOSEFA BELARMINO DE SOUSA']


<br>

## Lista de Endereços

In [9]:
from gerador_endereco import *

listas = get_list_ceps_bairros(estado='sp', municipio='piracicaba')
end = get_random_complete_address(random.choice(listas[0]))

Município: Piracicaba-SP
CEPs distintos: 257
Bairros distintos: 95
Endereço Aleatório:
Rua Capitão Emídio, 834 - São Dimas - Piracicaba, SP - CEP: 13416-040


<br>

## Tabelas

In [10]:
i = 1
data = []
while i <= 20:
    small_data = {
        'name': random.choice(list_nomes),
        'address': get_random_complete_address(random.choice(listas[0]))
    }
    data.append(small_data)
    time.sleep(2)
    i += 1

Endereço Aleatório:
Rua Capitão Ricardo Mazzoneto, 854 - Residencial Santo Antônio - Piracicaba, SP - CEP: 13402-570
Endereço Aleatório:
Praça Allan Kardec, 148 - Centro - Piracicaba, SP - CEP: 13400-020
Endereço Aleatório:
Travessa Professor Newton de Almeida Mello, 269 - Centro - Piracicaba, SP - CEP: 13400-400
Endereço Aleatório:
Rua 08 de Março, 916 - Ondas - Piracicaba, SP - CEP: 13403-774
Endereço Aleatório:
Praça Quinze de Novembro, 520 - Centro (Tupi) - Piracicaba, SP - CEP: 13428-408
Endereço Aleatório:
Rua Sargento Ivan Henrique da Silva, 94 - Pompéia - Piracicaba, SP - CEP: 13425-834
Endereço Aleatório:
Rua São Jorge, 689 - Santana - Piracicaba, SP - CEP: 13411-516
Endereço Aleatório:
Praça Antônio de Pádua Dutra, 891 - Centro - Piracicaba, SP - CEP: 13400-030
Endereço Aleatório:
Rua Antônio Carlos de Almeida, 276 - Santa Terezinha - Piracicaba, SP - CEP: 13411-191
Endereço Aleatório:
Praça Professor Sérgio Bonganhi, 51 - Terras de Piracicaba - Piracicaba, SP - CEP: 13403-84

In [11]:
# Dataframe
df = pd.DataFrame(data)
df.to_csv(
    os.path.join(data_path, 'nome_endereco.csv'),
    index=False,
)
# Results
df.head()

,name,address
0,MARCO AURELIO BOSELLI,{'logradouro': 'Rua Capitão Ricardo Mazzoneto'...
1,MARCELO CESAR VELASCO E SILVA,"{'logradouro': 'Praça Allan Kardec', 'numero':..."
2,LAVITOR BENVENUTTI,{'logradouro': 'Travessa Professor Newton de A...
3,LINO CARVALHO CERQUEIRA,"{'logradouro': 'Rua 08 de Março', 'numero': '9..."
4,MARCOS ALYSSANDRO SOARES DOS ANJOS,"{'logradouro': 'Praça Quinze de Novembro', 'nu..."


In [12]:
# Read Dataframe
df = pd.read_csv(os.path.join(data_path, 'nome_endereco.csv'))

def first_name(name):
    return (name.split(' ')[0]).title()

def last_name(name):
    return (name.split(' ')[-1]).title()

def username(first_name, last_name):
    return '{}{}'.format(first_name.lower(), last_name.lower())

df['first_name'] = df.apply(lambda x: first_name(x['name']), axis=1)
df['last_name'] = df.apply(lambda x: last_name(x['name']), axis=1)
df['username'] = df.apply(lambda x: username(x['first_name'], x['last_name']), axis=1)

df = df.drop_duplicates(subset='username')

display(df.head())
print(len(df))

,name,address,first_name,last_name,username
0,MARCO AURELIO BOSELLI,{'logradouro': 'Rua Capitão Ricardo Mazzoneto'...,Marco,Boselli,marcoboselli
1,MARCELO CESAR VELASCO E SILVA,"{'logradouro': 'Praça Allan Kardec', 'numero':...",Marcelo,Silva,marcelosilva
2,LAVITOR BENVENUTTI,{'logradouro': 'Travessa Professor Newton de A...,Lavitor,Benvenutti,lavitorbenvenutti
3,LINO CARVALHO CERQUEIRA,"{'logradouro': 'Rua 08 de Março', 'numero': '9...",Lino,Cerqueira,linocerqueira
4,MARCOS ALYSSANDRO SOARES DOS ANJOS,"{'logradouro': 'Praça Quinze de Novembro', 'nu...",Marcos,Anjos,marcosanjos


20


In [14]:
# Variables
list_courses = [1, 2, 3, 4, 5, 6, 7, 8, 9]
list_session = [1, 2, 3]

# Parameters
i = 1
pk = 10
data = []

# Loop
for index, row in df.iterrows():
    # Set Variables
    name = row['name']
    address = row['address']

    # dddd
    first_name = (name.split(' ')[0]).title()
    last_name = (name.split(' ')[-1]).title()
    username = '{}{}'.format(first_name.lower(), last_name.lower())
    sexo = personal.classify_name(name)
    course_id = random.choice(list_courses)
    session_year_id = random.choice(list_session)
    if sexo == 'Masculino':
        pic = "/media/menino.jpg"
    elif sexo == 'Feminino':
        pic = "/media/menina.jpg"
    else:
        print('deu ruim!')

    # Parte 1
    item_1 = {
        "model": "core.customuser",
        "pk": pk,
        "fields": {
            "password": "pbkdf2_sha256$260000$Tkf0RF5rJ2MsCoaOBNGl8x$HX1MdI6HNFymYFr+MmISOLkzpmeDUlU5xTLeduRL+z8=",
            "last_login": None,
            "is_superuser": False,
            "username": username,
            "first_name": first_name,
            "last_name": last_name,
            "email": "{}@gmail.com".format(username),
            "is_staff": False,
            "is_active": True,
            "user_type": "3",
            "groups": [],
            "user_permissions": []
        }
    }
    item_2 = {
        "model": "core.students",
        "pk": i,
        "fields": {
            "admin": pk,
            "gender": sexo,
            "profile_pic": pic,
            "address": address,
            "course_id": course_id,
            "session_year_id": session_year_id
        }
    }
    data.append(item_1)
    data.append(item_2)
    i += 1
    pk += 1

print(json.dumps(data, indent=2))

"Marco" é um nome "Masculino" com 99.13% de certeza
"Marcelo" é um nome "Masculino" com 99.03% de certeza
division by zero
"Lavitor" é um nome "Masculino" com 56.15% de certeza
"Lino" é um nome "Masculino" com 100.0% de certeza
"Marcos" é um nome "Masculino" com 99.13% de certeza
"Jussara" é um nome "Feminino" com 99.45% de certeza
"Lucia" é um nome "Feminino" com 99.35000000000001% de certeza
"Marcelo" é um nome "Masculino" com 99.03% de certeza
"Leonardo" é um nome "Masculino" com 99.15% de certeza
"Marcia" é um nome "Feminino" com 99.3% de certeza
"Lillian" é um nome "Feminino" com 100.0% de certeza
"Manoel" é um nome "Masculino" com 99.11% de certeza
"Lucilene" é um nome "Feminino" com 98.29% de certeza
"Manoel" é um nome "Masculino" com 99.11% de certeza
"Marcelina" é um nome "Feminino" com 100.0% de certeza
"Juliano" é um nome "Masculino" com 98.31% de certeza
division by zero
"Jumarino" é um nome "Masculino" com 27.27% de certeza
"Lays" é um nome "Feminino" com 100.0% de certeza

<br>

## Save Results

In [15]:
with open(os.path.join(data_path, 'students.json'), 'w') as outfile:
    json.dump(data, outfile)